In [48]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score
from isolated_ad_model.processing import PREPROCESS
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import SklearnClassifier, BlackBoxClassifier
from a2pm import A2PMethod
import joblib

In [ ]:
class AnomalyModelFactory:
    def __init__(self, model_recipe = None):
        self.model_recipe = model_recipe
        self.model = None

    def save_model(self, path="model.pkl"):
        joblib.dump(self.model, path)

    def load_model(self, path="model.pkl"):
        if not os.path.exists(path):
            raise FileNotFoundError(f"No model found at {path}. Please train the model first.")
        self.model = joblib.load(path)
        return self.model

    @staticmethod
    def get_scorer(true_labels):
        def scorer(estimator, X):
            pred = estimator.predict(X)
            pred = [1 if p == -1 else 0 for p in pred]
            return f1_score(true_labels, pred)
        return scorer
        
    def _get_iso_forest(self, training_data, true_anomalies):
        random_state = 4
        parameter = {'contamination': [of for of in np.arange(0.01, 0.5, 0.02)],
                     'n_estimators': [100*(i+1) for i in range(1, 10)],
                     'max_samples': [0.005, 0.01, 0.1, 0.15, 0.2, 0.3, 0.4]}
        cv = [(slice(None), slice(None))]
        scorer = self.get_scorer(true_anomalies)
        iso = IsolationForest(random_state=random_state, bootstrap=True, warm_start=False)
        model = RandomizedSearchCV(iso, parameter, scoring=scorer, cv=cv, n_iter=50)
        md = model.fit(training_data.values)
        return md.best_estimator_


    def build_model(self, training_data, true_anomalies):
        self.model = self._get_iso_forest(training_data, true_anomalies)
        return self.model
        

In [ ]:
class AnomalyRobustnessEvaluator:
    def __init__(self, model, training_data, true_anomalies):
        self.model = model
        self.training_data = training_data
        self.true_anomalies = true_anomalies
        
    @staticmethod
    def perform_a2pm_attack(pattern, training_data, model):
        a2pm_method = A2PMethod(pattern)
        a2pm_method.fit(training_data.values)
        
        raw_adv_training_data = a2pm_method.generate(model, training_data.values)

        return pd.DataFrame(raw_adv_training_data, columns=training_data.columns)
    
    @staticmethod
    def predict_wrapper(model, data):
        pred = model.predict(data.values)

        return [1 if p == -1 else 0 for p in pred]
    
    @staticmethod
    def get_hsja_predict(model):
        def hsja_predict(data):
            pred = model.predict(data)
            pred = [1 if p == -1 else 0 for p in pred]
            # print('data is ', data)
            # print('eye is ', np.eye(2)[pred])
            return np.eye(2)[pred] # The output needs to be one-hot
        return hsja_predict
    
    @staticmethod
    def print_metrics(pred):
        print(f"Total number of inlier = {sum([p==0 for p in pred])}")
        print(f"Total number of outlier = {sum([p==1 for p in pred])}")

    
    def test_a2pm(self):
        
        if self.model is None:
            raise ValueError("Model must be trained before testing attack")

        pred = self.predict_wrapper(self.model, self.training_data)
        f1_normal = f1_score(self.true_anomalies, pred)
        self.print_metrics(pred)

        # TODO: Figure out patterns - how to optimize them
        # pattern = (

        #         {
        #             "type": "interval",
        #             "features": None,
        #             "ratio": 0.1,
        #             "probability": 0.6,
        #             "momentum": 0.99
        #         },
        #     )

        pattern = (

                # First pattern to be applied: Interval
                {
                    "type": "interval",
                    "features": None,
                    "integer_features": None,
                    "ratio": 0.1,
                    "max_ratio": 0.3,
                    "missing_value": 0.0,
                    "probability": 0.6,
                },

                # # Second pattern to be applied: Combination
                # {
                #     "type": "combination",
                #     "features": None,
                #     "locked_features": None,
                #     "probability": 0.4,
                # },
            )
            
        adv_training_data = self.perform_a2pm_attack(pattern, self.training_data, self.model)

        adv_pred = self.predict_wrapper(self.model, adv_training_data)

        f1_adversarial = f1_score(self.true_anomalies, adv_pred)

        print("Regular f1_score ", f1_normal, ' vs. adversarial ', f1_adversarial, ' difference ', f1_normal- f1_adversarial)

        self.print_metrics(adv_pred)

    def test_hsja(self):
        
        if self.model is None:
            raise ValueError("Model must be trained before testing attack")

        pred = self.predict_wrapper(self.model, self.training_data)
        f1_normal = f1_score(self.true_anomalies, pred)
        self.print_metrics(pred)

        clip_values = (self.training_data.min().min(), self.training_data.max().max()) # Extract minimum and maximum values
        input_shape = (self.training_data.shape[1],)
        hsja_predict = self.get_hsja_predict(self.model)
        classifier = BlackBoxClassifier(predict_fn=hsja_predict,input_shape=input_shape,nb_classes=2,clip_values=clip_values)
        hsja = HopSkipJump(classifier=classifier)

        np_adv_data = hsja.generate(self.training_data.values[:20], max_iter=50, max_eval=10000, init_eval=100, verbose=True)
        self.adv_data = pd.DataFrame(np_adv_data, columns=self.training_data.columns)


        # self.training_data[:20].compare(self.adv_data)
        
        self.adv_pred = self.predict_wrapper(self.model, self.adv_data)

        # print("Original:", self.predict_wrapper(self.model, self.training_data[:5]))
        # print("Adversarial:", self.predict_wrapper(self.model, self.adv_data[:5]))

        f1_adversarial = f1_score(self.true_anomalies[:len(self.adv_pred)], self.adv_pred)
        print("Regular f1_score ", f1_normal, ' vs. adversarial ', f1_adversarial, ' difference ', f1_normal- f1_adversarial)
        self.print_metrics(self.adv_pred)


In [51]:
#config
use_cached_model = False
model_path = "model.pkl"

#obtain the dataset
dataset = pd.read_csv('../isolated_ad_model/ue.csv')
true_anomalies = dataset['Viavi.UE.anomalies']
ps = PREPROCESS(dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
ps.process()
training_data = ps.data

#obtain the model
model_factory = AnomalyModelFactory() # TODO: Add configuration from file

if use_cached_model:
    model = model_factory.load_model(model_path) 
else:
    model = model_factory.build_model(training_data, true_anomalies)
    model_factory.save_model(model_path)

#testing enviroment
evaluator = AnomalyRobustnessEvaluator(model, training_data, true_anomalies)

In [47]:
evaluator.test_a2pm()
evaluator.test_hsja()

Total number of inlier = 5700
Total number of outlier = 4300
Regular f1_score  0.6942341292952824  vs. adversarial  0.40031999999999995  difference  0.29391412929528243
Total number of inlier = 68
Total number of outlier = 9932
Total number of inlier = 5700
Total number of outlier = 4300


HopSkipJump:   5%|▌         | 1/20 [00:04<01:18,  4.14s/it]


KeyboardInterrupt: 

In [ ]:
# Calculate L2

print(evaluator.adv_data.compare(evaluator.training_data[:len(evaluator.adv_pred)]))

# l_inf = np.linalg.norm(evaluator.adv_data - evaluator.training_data, ord=np.inf, axis=1)
l2 = np.linalg.norm(evaluator.adv_data - evaluator.training_data[:len(evaluator.adv_pred)], ord=2, axis=1)
# l1 = np.linalg.norm(evaluator.adv_data - evaluator.training_data, ord=1, axis=1)

# print(l2)
# print(l1, l2, l_inf)